In [1]:
from pyspark.sql.types import *

In [2]:
customSchema = StructType([
    StructField("Primary_Index", IntegerType(), True), 
    StructField("Tweet_Id", StringType(), True), 
    StructField("Date_Text", StringType(), True),
    StructField("Flag", StringType(), True), 
    StructField("User", StringType(), True),
    StructField("Tweet_Text", StringType(), True)])

df = spark.read.load('hdfs://localhost:9000/CA2/ProjectTweets.csv', format="csv", header="False", sep=',', schema=customSchema)


In [3]:
##Remove commas from the Tweet Text field
##First test an example

In [4]:
filtered_df = df.filter(df["Tweet_Id"] == '1467811594')
# Select only the "Tweet_Text" column from the filtered DataFrame
result = filtered_df.select("Tweet_Text")

# Show the content of column "Tweet_Text"
result.show(truncate=False)

+---------------------------------------------------------------------------------------------------+
|Tweet_Text                                                                                         |
+---------------------------------------------------------------------------------------------------+
|@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?|
+---------------------------------------------------------------------------------------------------+



In [5]:
##Then strip the commas out

In [6]:
from pyspark.sql.functions import regexp_replace
# Remove commas from the "Tweet_Text" column

filtered_df = df.filter(df["Tweet_Id"] == '1467811594')

filtered_df = filtered_df.withColumn("Tweet_Text", regexp_replace(filtered_df["Tweet_Text"], ",", ""))

# Select only the "Tweet_Text" column from the filtered DataFrame
result = filtered_df.select("Tweet_Text")

# Show the content of column "Tweet_Text"
result.show(truncate=False)

+------------------------------------------------------------------------------------------------+
|Tweet_Text                                                                                      |
+------------------------------------------------------------------------------------------------+
|@LOLTrish hey  long time no see! Yes.. Rains a bit only a bit  LOL  I'm fine thanks  how's you ?|
+------------------------------------------------------------------------------------------------+



In [7]:
##Now do it with all the rows

In [8]:
df = df.withColumn("Tweet_Text", regexp_replace(df["Tweet_Text"], ",", ""))

In [9]:
filtered_df = df.filter(df["Tweet_Id"] == '1467811594')
# Select only the "Tweet_Text" column from the filtered DataFrame
result = filtered_df.select("Tweet_Text")

# Show the content of column "Tweet_Text"
result.show(truncate=False)

+------------------------------------------------------------------------------------------------+
|Tweet_Text                                                                                      |
+------------------------------------------------------------------------------------------------+
|@LOLTrish hey  long time no see! Yes.. Rains a bit only a bit  LOL  I'm fine thanks  how's you ?|
+------------------------------------------------------------------------------------------------+



In [10]:

# Calculate the max length of each column
from pyspark.sql.functions import max, length
max_lengths = []

for column_name in df.columns:
    max_length = df.agg(max(length(column_name))).collect()[0][0]
    max_lengths.append((column_name, max_length))

# Display the results
for col, max_len in max_lengths:
    print(f"Max length of {col}: {max_len}")

Max length of Primary_Index: 7
Max length of Tweet_Id: 10
Max length of Date_Text: 28
Max length of Flag: 8
Max length of User: 15
Max length of Tweet_Text: 374


In [11]:
from pyspark.sql.functions import col, sum

# Count null values in each column
null_counts = []

for column_name in df.columns:
    count = df.select(column_name).where(col(column_name).isNull()).count()
    null_counts.append((column_name, count))

# Display the results
for col, count in null_counts:
    print(f"Number of null values in {col}: {count}")

Number of null values in Primary_Index: 0
Number of null values in Tweet_Id: 0
Number of null values in Date_Text: 0
Number of null values in Flag: 0
Number of null values in User: 0
Number of null values in Tweet_Text: 0


In [12]:
from pyspark.sql.functions import countDistinct

# Calculate the number of unique values in each column
unique_counts = []

for column_name in df.columns:
    count = df.agg(countDistinct(column_name)).collect()[0][0]
    unique_counts.append((column_name, count))

# Display the results
for col, count in unique_counts:
    print(f"Number of unique values in {col}: {count}")

Number of unique values in Primary_Index: 1600000
Number of unique values in Tweet_Id: 1598315
Number of unique values in Date_Text: 774363
Number of unique values in Flag: 1
Number of unique values in User: 659775
Number of unique values in Tweet_Text: 1581312


In [14]:
from pyspark.sql.functions import col
# Find and display an example record with duplicated "Tweet_Id"
duplicate_tweet_id_example = df.groupBy("Tweet_Id").count().filter(col("count") > 1).limit(1)

# Display the result
duplicate_tweet_id_example.show()

+----------+-----+
|  Tweet_Id|count|
+----------+-----+
|1469531660|    2|
+----------+-----+



In [15]:
# Find and display all records with the Tweet_Id "1469531660"
matching_records = df.filter(col("Tweet_Id") == "1469531660")

# Display the matching records
matching_records.show(truncate=False)

+-------------+----------+----------------------------+--------+------------+-----------------------------------------------------------------+
|Primary_Index|Tweet_Id  |Date_Text                   |Flag    |User        |Tweet_Text                                                       |
+-------------+----------+----------------------------+--------+------------+-----------------------------------------------------------------+
|6730         |1469531660|Tue Apr 07 06:39:53 PDT 2009|NO_QUERY|appleaddicto|Company blocked Twitter today  oh well i still have it on mobile |
|809639       |1469531660|Tue Apr 07 06:39:53 PDT 2009|NO_QUERY|appleaddicto|Company blocked Twitter today  oh well i still have it on mobile |
+-------------+----------+----------------------------+--------+------------+-----------------------------------------------------------------+



In [16]:
# Create a DataFrame with unique records
unique_records_df = df.dropDuplicates(["Tweet_Id", "Date_Text", "Flag", "User", "Tweet_Text"])


In [17]:
# Calculate the number of unique values in each column
unique_counts = []

for column_name in df.columns:
    count = unique_records_df.agg(countDistinct(column_name)).collect()[0][0]
    unique_counts.append((column_name, count))

# Display the results
for col, count in unique_counts:
    print(f"Number of unique values in {col}: {count}")

2023-10-28 14:11:52,875 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2023-10-28 14:11:53,219 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2023-10-28 14:11:53,366 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2023-10-28 14:11:53,838 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Number of unique values in Primary_Index: 1598315
Number of unique values in Tweet_Id: 1598315
Number of unique values in Date_Text: 774363
Number of unique values in Flag: 1
Number of unique values in User: 659775
Number of unique values in Tweet_Text: 1581312


In [19]:
from pyspark.sql.functions import col
# Find and display all records with the Tweet_Id "1469531660"
matching_records = unique_records_df.filter(col("Tweet_Id") == "1469531660")

# Display the matching records
matching_records.show(truncate=False)

2023-10-28 14:14:49,697 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2023-10-28 14:14:49,743 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2023-10-28 14:14:49,979 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2023-10-28 14:14:50,055 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------------+----------+----------------------------+--------+------------+-----------------------------------------------------------------+
|Primary_Index|Tweet_Id  |Date_Text                   |Flag    |User        |Tweet_Text                                                       |
+-------------+----------+----------------------------+--------+------------+-----------------------------------------------------------------+
|6730         |1469531660|Tue Apr 07 06:39:53 PDT 2009|NO_QUERY|appleaddicto|Company blocked Twitter today  oh well i still have it on mobile |
+-------------+----------+----------------------------+--------+------------+-----------------------------------------------------------------+



In [20]:
df = unique_records_df

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql import SparkSession

# Tokenize the "Tweet_Text" column
tokenizer = Tokenizer(inputCol="Tweet_Text", outputCol="words")
filtered_df = tokenizer.transform(filtered_df)

# Remove stopwords from the "words" column
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_text")
filtered_df = stopwords_remover.transform(filtered_df)

# Show the modified DataFrame
filtered_df.show()

In [ ]:
result = filtered_df.select("filtered_text")

# Show the content of column "Tweet_Text"
result.show(truncate=False)